In [2]:
import os
import json

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_count = torch.cuda.device_count()
print(f"Using device: {torch.cuda.current_device()}, Number of GPUs: {device_count}")

Using device: 0, Number of GPUs: 2


In [4]:
from langchain.schema import Document
from langchain.document_loaders import Docx2txtLoader
import re

doc_path = "./data/국어 지식 기반 생성(RAG) 참조 문서.docx"
loader = Docx2txtLoader(doc_path)

docs = loader.load()

# 모든 문서 텍스트를 하나로 결합
full_text = "\n".join([doc.page_content for doc in docs])

# 3개 이상 개행을 2개로 줄이기
full_text = re.sub(r'\n{3,}', '\n\n', full_text)

# 타이틀과 섹션 분리
titles = re.findall(r"<([^>]+)>", full_text)
sections = re.split(r"<[^>]+>", full_text)[1:]

# Document 객체 생성
rule_docs = [
    Document(
        page_content=content.strip(),
        metadata={
            "title": title.strip(),
            "category": title.split("-")[0].strip(),
            "length": len(content.strip())
        }
    )
    for title, content in zip(titles, sections)
]

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
import torch


# jhgan/ko-sroberta-multitask : 3998MB
# jhgan/ko-sbert-nli : 4066MB
# jhgan/ko-sroberta-nli : 4012MB
# jhgan/ko-sroberta-=multitask : 4048MB

# intfloat/multilingual-e5-large-instruct : 17602MB
# intfloat/multilingual-e5-base : 7908MB
# intfloat/multilingual-e5-small: 3950MB

# Alibaba-NLP/gte-multilingual-base : 10402MB

# infly/inf-retriever-v1-1.5b : memory error

#kakao kanana nano 2.1B embedding

# klue/bert-base => 867MB


embedding_model = HuggingFaceEmbeddings(
    model="nlpai-lab/KURE-v1",
    model_kwargs={
        "device": "cuda",
        "model_kwargs": {
            "torch_dtype": torch.bfloat16
        }
    },
    encode_kwargs= {
        "normalize_embeddings": True
    },
    query_encode_kwargs={
        "normalize_embeddings": True
    }
)

In [6]:
vectorstore = FAISS.from_documents(rule_docs, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel, PeftConfig
import torch

device = "cuda"
base_model_id = "LGAI-EXAONE/EXAONE-4.0-32B"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map=device,
    trust_remote_code=True,
    quantization_config=bnb_config
)

/home/dsc_capstone/miniconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-28 08:17:12.380132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753690632.400755 1388490 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753690632.407083 1388490 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753690632.423664 1388490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

In [7]:
from langchain_huggingface import HuggingFacePipeline


generator = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    repetition_penalty=1.05,
    temperature=0.5,
    top_p=0.95,
    return_full_text=False,
    use_cache=False
)

llm = HuggingFacePipeline(pipeline=generator)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [8]:
from langchain.prompts import ChatPromptTemplate

c_type_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
    너는 한국어 문법 교정 시스템에서 retriever에게 전달할 질의를 만드는 전문가야.

    입력 문장은 중괄호 {{ }} 안에 두 표현이 들어 있는 **선택형 문장**이야.

    [지침]
    - 중괄호 안의 표현 둘이 어떤 문법 규칙에 따라 구별되는지를 설명하는 **질문 문장**으로 재작성하라.
    - 문장의 의미는 유지하되, 문법 정보(띄어쓰기, 어미 활용, 문장 부호, 한국어 맞춤법 등)가 드러나는 자연어 문장으로 확장하라.
    - **오직 재작성된 질의 한 문장만 출력하라.**

    [예시]
    입력: "나는 그를 본 적이 있음을 {{기억해냈다/기억해 냈다}}."
    출력: '기억하다'와 '–아/어 내다'가 결합된 경우 보조 용언 앞에 띄어쓰는 것이 맞는지에 대한 규범을 확인하는 질의이다.
    """
    ),
    ("human", "입력: {query}\n출력:")
])

In [9]:
r_type_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
    너는 한국어 문법 교정 시스템에서 retriever에게 전달할 질의를 만드는 전문가야.

    입력 문장은 큰따옴표 안에 포함된 **교정형 문장**이야.

    [지침]
    - 문장에 존재할 수 있는 **어문 규범 위반의 가능성**을 중심으로, 문법 지식에 기반한 질의로 재작성하라.
    - 문장의 의미는 유지하되, 문법/표기상의 문제가 의심되는 표현이 어떤 규범을 위반할 수 있는지를 설명하는 방향으로 재구성하라.
    - **오직 재작성된 질의 한 문장만 출력하라.**

    [예시]
    입력: "또한 갸름한 얼굴에 초승달 같은 눈썹, 가늘고 긴 눈, 오똑한 코, 작고 예쁜 입 등 섬세한 이목구비가 우아한 인상을 준다."
    출력: '오똑하다'와 '오뚝하다' 중 어떤 표현이 표준어로 적절한지, 음운 변화나 표준어 규정에 따라 판단하는 질의이다.
    """
    ),
    ("human", "입력: {query}\n출력:")
])

In [10]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     "너는 문장 속의 어문 규범 위반 요소를 추출하는 한국어 문법 보조자야.\n\n"
     "지침:\n"
     "- 큰따옴표 안에 있는 문장만 분석한다.\n"
     "- 한국어 맞춤법, 띄어쓰기, 어미 활용, 문장 부호 등의 규범 위반 요소를 한 가지만 식별한다.(해당 문장은 오류가 무조건 포함되어 있음)\n"
     "- 잘못된 부분과 위반된 규범 요소를 한 두 문장으로 설명한다.\n"
    ),
    ("human", "입력: {query}\n출력:")
])


In [ ]:
import re


def expand_query(query: str, llm, prompt_type):
    if prompt_type == "선택형":
        prompt = c_type_prompt.format_messages(query=query)
    elif prompt_type == "교정형":
        prompt = r_type_prompt.format_messages(query=query)
    else:
        raise ValueError("Wrong prompt type!")
    
    formatted_prompt = llm.pipeline.tokenizer.apply_chat_template(
        [
            {"role": "system", "content": prompt[0].content},
            {"role": "user", "content": prompt[1].content}
        ],         
        enable_thinking=False,
        add_generation_prompt=True,
        tokenize=False)
    result = llm(formatted_prompt).strip()
    return result.split("\n")[-1].strip()


In [12]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
import json
import random

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    """참조문서:
{context}

────────────────── [예시] ──────────────────
[질문] "오늘은 날씨가 {{푹하다/푸카다}}." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.
[답변] "오늘은 날씨가 푹하다."가 옳다. '-하다'나 '-없다'가 붙어서 된 용언은 그 '-하다'나 '-없다'를 밝혀 적는 것이 원칙이다. 따라서 '푸카다'는 비표준어이다.

[질문] "나는 그를 본 적이 있음을 {{기억해냈다/기억해 냈다}}." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.
[답변] "나는 그를 본 적이 있음을 기억해 냈다."가 옳다. '기억하다'는 3음절 파생어이므로 보조 용언 '내다'와는 띄어 써야 한다.

────────────────── [지침] ──────────────────
1. [답변]의 첫 문장은 질문 속 **큰따옴표 문장을 교정**해 제시하며, 끝을 **"가 옳다." 또는 "이 옳다."**로 마무리한다.
2. 이어지는 문장은 **실제 바뀐 어절 중심으로 문법적 설명을 간결하게 기술**한다. 1~2문장 이내로 작성한다.
3. 참조문서의 설명이 명백히 어문 규범과 모순되거나 오류가 있는 경우에는 **정확한 어문 규범에 따라 판단하고, 참조문서를 무시할 수 있음**
4. **'참조문서에 따라', '참조문서에 의하면'** 같은 말은 [답변]에 절대 포함하지 않는다.

────────────────── [출력 형식] ──────────────────
[답변] "정답 문장"가/이 옳다. 설명 문장.
"""
)


user_prompt = HumanMessagePromptTemplate.from_template(
    "[질문] {input}\n[답변]"
)


final_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, user_prompt]
)


In [14]:
def format_context(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:
from langchain_core.runnables import RunnableLambda, RunnableMap

generate_messages = RunnableLambda(
    lambda x: final_prompt.format_messages(
        input= x["question"],
        context = format_context(retriever.get_relevant_documents(expand_query(x["question"], llm, x["question_type"])))
    )
)

In [16]:
convert_to_prompt = RunnableLambda(
    lambda messages: llm.pipeline.tokenizer.apply_chat_template(
        [
            {"role": "system", "content": messages[0].content},
            {"role": "user", "content": messages[1].content}
        ],
        enable_thinking=False,
        add_generation_prompt=True,
        tokenize=False
    )
)

In [ ]:
import re

def post_process(text: str):
    split_patterns = ['가 옳다.', '이 옳다.']
    
    for pattern in split_patterns:
        if pattern in text:
            split_idx = text.find(pattern)
            answer = text[:split_idx].strip()
            reason = text[split_idx + len(pattern):].strip()
            matched_pattern = pattern
            break
    else:
        print(f"[post_process 경고] 형식 미일치: {text}")
        return text.strip()

    has_left = answer.startswith('"')
    has_right = answer.endswith('"')

    if has_left and has_right:
        fixed_answer = answer
    elif has_left and not has_right:
        fixed_answer = answer + '"'
    elif not has_left and has_right:
        fixed_answer = '"' + answer
    else:
        fixed_answer = '"' + answer + '"'

    return f"{fixed_answer.strip()}{matched_pattern} {reason.strip()}"


In [18]:
import re

def generate_with_qwen(prompt: str):
    output = llm(prompt).strip()

    # [답변] 태그 이후 문장 추출
    match = re.search(r"\[답변\](.+)", output, flags=re.DOTALL)
    if match:
        result = match.group(1).strip().replace("\n", "").strip()
        return post_process(result)

    # fallback: 마지막 줄 기준
    last_line = output.split("\n")[-1]
    print(output)
    result = last_line.replace("[답변]", "").replace("\n", "").strip()
    return post_process(result)


generate_runnable = RunnableLambda(generate_with_qwen)

In [19]:
qwen_fewshot_runnable = (
    generate_messages |
    convert_to_prompt |
    generate_runnable 
)

In [23]:
sample = {
    "id": "626",
    "input": {
      "question_type": "교정형",
      "question": "다음 문장이 어문 규범에 부합하도록 문장 부호를 추가하고, 그 이유를 설명하세요.\n― 식물 관찰하기( )기르기"
    }
}
payload = {
    "question_type": sample["input"]["question_type"],
    "question": sample["input"]["question"],
}
qwen_fewshot_runnable.invoke(payload)


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


KeyboardInterrupt: 

In [20]:
from tqdm import tqdm

with open("./data/korean_language_rag_V1.0_dev.json") as fr:
    dev_data = json.load(fr)

result = []
for sample in tqdm(dev_data):
    payload = {
        "question_type": sample["input"]["question_type"],
        "question": sample["input"]["question"],
    }
    output = qwen_fewshot_runnable.invoke(payload)
    sample.update({"output": {"answer": output}})
    result.append(sample)
result[:10]    

  0%|          | 0/127 [00:00<?, ?it/s]/tmp/ipykernel_1388490/3056363271.py:20: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(formatted_prompt).strip()
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_1388490/4023137386.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = format_context(retriever.get_relevant_documents(expand_query(x["question"], llm, x["question_type"])))
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  1%|          | 1/127 [00:56<1:58:46, 56.56s/it]The following generation flags are not valid and may be ig

"나는 그를 본 적이 있음을 기억해 냈다."가 옳다. '기억해냈다'는 '기억하다'와 '내다'가 결합된 형태로, 보조 용언은 본 용언과 띄어 쓰는 것이 원칙이다.


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  2%|▏         | 2/127 [02:47<3:04:53, 88.75s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  2%|▏         | 3/127 [03:47<2:36:05, 75.53s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  3%|▎         | 4/127 [04:39<2:15:21, 66.03s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more deta

"[부득이하게] 늦은 시간에 전화를 드렸습니다."가 옳다. '-하게'는 형용사 '부득이하다'에서 파생된 어미로, '부득이하게'가 표준어이다.


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 14%|█▍        | 18/127 [18:20<1:53:56, 62.72s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 15%|█▍        | 19/127 [19:51<2:08:13, 71.24s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 16%|█▌        | 20/127 [20:49<1:59:50, 67.20s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more d

...

"이제 와서 고백건대 나는 그날 동진이를 만나지 않았다."가 옳다. '고백하다'의 활용형인 '고백건대'는 어간과 어미가 결합된 형태로, 보조 용언이 아닌 본용언으로 사용되었기 때문에 붙여 써야 한다.


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 46%|████▋     | 59/127 [1:01:38<1:09:31, 61.34s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 47%|████▋     | 60/127 [1:02:23<1:03:04, 56.48s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 48%|████▊     | 61/127 [1:03:30<1:05:36, 59.65s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for 

"폐품으로 무언가를 만드는 게 방학 숙제야."가 옳다. '폐품'은 본말이 '폐기물'이므로 본말을 밝혀 적는 것이 원칙이다.


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 95%|█████████▌| 121/127 [2:00:09<04:34, 45.77s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 96%|█████████▌| 122/127 [2:00:48<03:38, 43.70s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
 97%|█████████▋| 123/127 [2:01:42<03:07, 46.93s/it]The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for mor

[{'id': '623',
  'input': {'question_type': '선택형',
   'question': '"나는 그를 본 적이 있음을 {기억해냈다/기억해 냈다}." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.'},
  'output': {'answer': '"나는 그를 본 적이 있음을 기억해 냈다."가 옳다. \'기억해냈다\'는 \'기억하다\'와 \'내다\'가 결합된 형태로, 보조 용언은 본 용언과 띄어 쓰는 것이 원칙이다.'}},
 {'id': '624',
  'input': {'question_type': '교정형',
   'question': '다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그 이유를 설명하세요.\n"오늘은 퍼즐 마추기를 해 볼 거예요."'},
  'output': {'answer': '"오늘은 퍼즐 맞추기를 해 볼 거예요."가 옳다. \'맞추기\'는 \'맞추다\'의 어간에 \'-기\'가 붙어 파생된 명사이므로 붙여 써야 하며, \'마추기\'는 비표준어이다.'}},
 {'id': '625',
  'input': {'question_type': '선택형',
   'question': '"오늘은 날씨가 {푹하다/푸카다}." 가운데 올바른 것을 선택하고, 그 이유를 설명하세요.'},
  'output': {'answer': '"오늘은 날씨가 푹하다."가 옳다. \'-하다\'가 붙어 형성된 용언은 \'-하다\'를 밝혀 적는 것이 원칙이므로 \'푸카다\'는 비표준어이다.'}},
 {'id': '626',
  'input': {'question_type': '교정형',
   'question': '다음 문장이 어문 규범에 부합하도록 문장 부호를 추가하고, 그 이유를 설명하세요.\n― 식물 관찰하기( )기르기'},
  'output': {'answer': '"식물 관찰하기 ― 기르기"가 옳다. 참조문서 (2)에 따라 열거된 항목 중 선택 가능함을 나타내기 위해 줄표(―)를 사용하며, 줄표 앞뒤는 붙여 쓰는 것이 

In [21]:
with open("./data/test_eval/exaone_rag_dev.json", "w") as fw:
    json.dump(result, fw, ensure_ascii=False, indent=2)

In [ ]:
results = []
for i, sample in enumerate(test_data):
    payload = {
        "question_type": sample["input"]["question_type"],
        "question": sample["input"]["question"],
    }
    parsed = rag_chain.invoke(payload)
    if i % 10 == 0:
        print(f"{i + 1}번째 질문: {payload['question']}")
        print(f"{i + 1}번째 context: {retriever.get_relevant_documents(payload['question'])}")
        print(f"{i + 1}번째 예측: {parsed}")
    results.append({
        "id": sample["id"],
        "input": sample["input"],
        "output": {"answer": parsed},
    })

print(results)